In [1]:
# Dependencies and Setup
import pathlib
import pandas as pd

# Creates a path for the raw data file 
school_rdata = pathlib.Path("Resources/schools_complete.csv")
student_rdata = pathlib.Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_rdata)
student_data = pd.read_csv(student_rdata)

# Combines the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary


##### Total Number of Schools in a District

In [2]:
#Sets the index to type 
District_df = school_data_complete.set_index("type")

In [3]:
#Filters type to look at District Schools only & returns school names without duplicates
Distr_schools = District_df.loc["District", "school_name"].unique()

#Returns the total counts for District only 
total_District_schools = len(Distr_schools)

##### Total Number of Students in a District

In [4]:
#Filters type to look at District Schools only & returns size (# of students in each school) without duplicates
Distr_students = District_df.loc["District","size"].unique()

#Calculates the total number of students in all District school 
total_District_students = Distr_students.sum()

##### Total District Schools Budget

In [5]:
#Filters District row and budget column only
Dist_Budget = District_df.loc["District", "budget"].unique()

#Calculates the total budget for District 
total_District_budget = Dist_Budget.sum()

##### District Average Math and Reading Scores

In [6]:
#Creates a new df that is filtered by District row & Math/Reading scores columns 
#Know data is correct if row counts matches total number of District student
District_Scores_df = District_df.loc[[ "District"], ["math_score", "reading_score"]]

In [7]:
#Calculates the average of Math scores only  
Math_Avg = District_Scores_df["math_score"].mean()

In [8]:
#Calculates the average of Reading scores only 
Reading_Avg = District_Scores_df["reading_score"].mean()

##### District Students with Passing Math and Reading Scores 

In [9]:
#Defines, filters passing MATH scores (only math scores >= 70 in the df) & creates new filtered df
passing_Mscore = District_Scores_df["math_score"] >= 70
passing_math_df = District_Scores_df.loc[passing_Mscore, ["math_score"]]

#Returns the number of students with passing Math scores
Math_passers = passing_math_df["math_score"].count()

In [10]:
#Defines, filters passing READING scores (only reading scores >= 70 in the df) & creates new filtered df 
passing_Rscore = District_Scores_df["reading_score"] >= 70
passing_reading_df = District_Scores_df.loc[passing_Rscore, ["reading_score"]]

#Returns the number of students with passing Reading scores
Reading_passers = passing_reading_df["reading_score"].count()

In [11]:
#Defines and filters number of students passing MATH AND READING (only math & reading scores >= 70 in the df) then creating a new filtered df
pass_math_AND_reading = passing_Mscore & passing_Rscore
studentspass_MandR_df = District_Scores_df.loc[pass_math_AND_reading, :]

#Returns the number of students with passing Math & Reading scores
Math_Reading_passers = studentspass_MandR_df["math_score"].count()

##### Percentage of District Students Passing Math & Reading 

In [12]:
#Calculates percentage of students with MATH Passing scores 
pct_Math_passing = (Math_passers / total_District_students) * 100

In [13]:
#Calculates percentage of students with READING Passing scores 
pct_Reading_passing = (Reading_passers / total_District_students) * 100

In [14]:
#Calculates percentage of students with MATH & READING Passing scores 
pct_MathReading_passing = (Math_Reading_passers / total_District_students) * 100

In [15]:
#Creates a dictionary of District results
District = {
    "Total Schools": [total_District_schools], 
    "Total Students": [total_District_students],
    "Total Budget": [total_District_budget],
    "Average Math Score": [Math_Avg],
    "Average Reading Score": [Reading_Avg],
    "% Math Passing": [pct_Math_passing],
    "% Reading Passing": [pct_Reading_passing],
    "% Overall Passing": [pct_MathReading_passing]
}
#Creates summary dataframe that includes dictionary of District results
District_Summary_df = pd.DataFrame(District)

#Fixes formatting of total students and total budget values
District_Summary_df['Total Budget'] = District_Summary_df['Total Budget'].map('${:,.2f}'.format)
District_Summary_df['Total Students'] = District_Summary_df['Total Students'].map('{:,.0f}'.format)

District_Summary_df.index = [' ']
District_Summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Math Passing,% Reading Passing,% Overall Passing
,7,"26,976","$17,347,923.00",76.987026,80.962485,66.518387,80.905249,53.695878


## School Summary

In [16]:
#Creates a dataframe with selected columns of interest from the merged data
allschools_df = school_data_complete[["Student ID","school_name", "type", "size", "budget", "math_score", "reading_score"]]

##### Per Student Budget 

In [17]:
#Creates a new column in the df above that includes the calculated amount of budget per student 
allschools_df["Per Student Budget"] = allschools_df["budget"] / allschools_df["size"]

<ipython-input-17-948b583e3e32>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allschools_df["Per Student Budget"] = allschools_df["budget"] / allschools_df["size"]


##### School Students with Passing Scores in Math, Reading and Overall (Math & Reading)

In [18]:
#Total number of students (to be used in calculating % Math, % Reading & Overall Passing)
#Creates a df grouped by school name with total number of students 
student_total = allschools_df.groupby(['school_name'])["Student ID"].count()

In [19]:
#Filters df with MATH passers (only include math scores >= 70) from all schools
schoolMath_passing = allschools_df.loc[allschools_df["math_score"] >= 70]

#Creates a df grouped by school name with total number of students, who PASSED MATH ONLY
group_MathPassers = schoolMath_passing.groupby(['school_name'])['Student ID'].count()

In [20]:
#Filters df with READING passers (only include reading scores >= 70) from all schools
schoolReading_passing = allschools_df.loc[allschools_df["reading_score"] >= 70]

#Creates a df grouped by school name with total number of students, who PASSED READING ONLY
group_ReadingPassers = schoolReading_passing.groupby(['school_name'])['Student ID'].count()

In [21]:
#Creates a variable that includes students with MATH and READING passing scores
MathReading_pass_scores = (allschools_df["math_score"] >= 70) & (allschools_df["reading_score"] >= 70)

#Filters df with MATH & READING passers (only include math & reading scores >= 70) from all schools
schoolMathReading_passing = allschools_df.loc[MathReading_pass_scores]

#Creates a df grouped by school name with total number of students, who PASSED MATH & READING 
group_MathReading_Passers = schoolMathReading_passing.groupby(['school_name'])['Student ID'].count()

##### Passing Rates for Math, Reading & Overall

In [22]:
#Calculate Math, Reading & Overall passing rates
pct_schoolMath = (group_MathPassers / student_total)*100
pct_schoolReading = (group_ReadingPassers / student_total)*100
pct_schoolMathReading = (group_MathReading_Passers / student_total)*100

In [23]:
#Set up School Summary Dataframe & clean up stage
#Uses the base df created set in the very beginning for school summary only (allschools_df)
school_groups_df = allschools_df.groupby(["school_name"]).mean()

#Adds type back (disappeared after the groupby above since non-numeric column)
school_groups_df["School Type"] = allschools_df.groupby(["school_name"])['type'].min()

#Add columns for Passing Rates
school_groups_df["% Math Passing"] = pct_schoolMath
school_groups_df["% Reading Passing"] = pct_schoolReading
school_groups_df["% Overall Passing"] = pct_schoolMathReading

#Drops school ID (only needed to clarify that we are aggregating students)
school_groups_df = school_groups_df.drop(["Student ID"], axis=1)

#Renames some columns
school_groups_df = school_groups_df.rename(columns={"size":"Total Students",
                                          "budget":"Total School Budget",
                                          "math_score":"Average Math Score",
                                          "reading_score":"Average Reading Score"})
#Removes index title 
school_groups_df.index.names = ['']

#Fixes formatting of total budget, total students and per student budget values
school_groups_df['Total School Budget'] = school_groups_df['Total School Budget'].map('${:,.2f}'.format)
school_groups_df['Per Student Budget'] = school_groups_df['Per Student Budget'].map('{:.2f}'.format)


In [24]:
#Rearranged columns for final summary
school_summary_df = school_groups_df[["School Type", "Total Students", 
                  "Total School Budget", "Per Student Budget", 
                  "Average Math Score", "Average Reading Score", 
                  "% Math Passing", "% Reading Passing", 
                  "% Overall Passing"]]

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Math Passing,% Reading Passing,% Overall Passing
,,,,,,,,,
Bailey High School,District,4976.0,"$3,124,928.00",628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858.0,"$1,081,356.00",582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949.0,"$1,884,411.00",639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739.0,"$1,763,916.00",644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468.0,"$917,500.00",625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635.0,"$3,022,020.00",652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427.0,"$248,087.00",581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917.0,"$1,910,635.00",655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761.0,"$3,094,650.00",650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

In [25]:
#Sorts and displays the top five performing schools by % overall passing
topfive_Overall_df = school_summary_df.sort_values('% Overall Passing', ascending=False).head()
topfive_Overall_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Math Passing,% Reading Passing,% Overall Passing
,,,,,,,,,
Cabrera High School,Charter,1858.0,"$1,081,356.00",582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635.0,"$1,043,130.00",638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468.0,"$917,500.00",625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283.0,"$1,319,574.00",578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962.0,"$585,858.00",609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [26]:
#Sorts and displays the five worst-performing schools by % overall passing
bottomfive_Overall_df = school_summary_df.sort_values('% Overall Passing', ascending=True).head()
bottomfive_Overall_df 

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Math Passing,% Reading Passing,% Overall Passing
,,,,,,,,,
Rodriguez High School,District,3999.0,"$2,547,363.00",637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949.0,"$1,884,411.00",639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917.0,"$1,910,635.00",655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635.0,"$3,022,020.00",652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761.0,"$3,094,650.00",650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [27]:
#Uses merged data as base df
#Narrows down df by selecting columns of interest and set index to the name of school
bygrade_df = school_data_complete[["school_name", "grade", "math_score", "reading_score"]]
indexed_bygrade = bygrade_df.set_index(["school_name"])

In [28]:
#Filters df for 9th grade only
nineth_students = (indexed_bygrade["grade"] == "9th") 
nineth_scores = indexed_bygrade.loc[nineth_students]

#Creates series for average math scores for 9th graders
avgMath_ninth = nineth_scores.groupby(["school_name"])['math_score'].mean()

In [29]:
#Filters df for 10th grade only
tenth_students = (indexed_bygrade["grade"] == "10th") 
tenth_scores = indexed_bygrade.loc[tenth_students]

#Creates series for average math scores for 10th graders
avgMath_tenth = tenth_scores.groupby(["school_name"])['math_score'].mean()

In [30]:
#Filters df for 11th grade only
eleventh_students = (indexed_bygrade["grade"] == "11th")
eleventh_scores = indexed_bygrade.loc[eleventh_students]

#Creates series for average math scores for 11th graders
avgMath_eleventh = eleventh_scores.groupby(["school_name"])['math_score'].mean()

In [31]:
#Filters df for 12th grade only
twelveth_students = (indexed_bygrade["grade"] == "12th")
twelveth_scores = indexed_bygrade.loc[twelveth_students]

#Creates series for average math scores for 12th graders
avgMath_twelveth = twelveth_scores.groupby(["school_name"])['math_score'].mean()

In [32]:
#Creates final dataframe summary for Math Scores by Grade
Math_byGrade = pd.DataFrame({
    "9th": avgMath_ninth, 
    "10th": avgMath_tenth,
    "11th": avgMath_eleventh,
    "12th": avgMath_twelveth
})
Math_byGrade.index.names = ['']
Math_byGrade

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [33]:
#Creates series for average Reading scores for 9th graders
avgReading_ninth = nineth_scores.groupby(["school_name"])['reading_score'].mean()

In [34]:
#Creates series for average Reading scores for 10th graders
avgReading_tenth = tenth_scores.groupby(["school_name"])['reading_score'].mean()

In [35]:
#Creates series for average Reading scores for 11th graders
avgReading_eleventh = eleventh_scores.groupby(["school_name"])['reading_score'].mean()

In [36]:
#Creates series for average Reading scores for 12th graders
avgReading_twelveth = twelveth_scores.groupby(["school_name"])['reading_score'].mean()

In [37]:
#Creates final dataframe summary for Reading Scores by Grade
Reading_byGrade = pd.DataFrame({
    "9th": avgReading_ninth, 
    "10th": avgReading_tenth,
    "11th": avgReading_eleventh,
    "12th": avgReading_twelveth
})
Reading_byGrade.index.names = ['']
Reading_byGrade

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [38]:
#Uses school_summary_df as base df and deletes columns that are not needed
school_spending_df = school_summary_df.drop(["Total Students", "Total School Budget"], axis=1)

In [39]:
#Creates a series that contains the list of bin data (Per Student Budget)
#Convert values from strings to float since values need to be a float to bin
cut_series = school_spending_df["Per Student Budget"].astype(float)

#Creates list of breakpoints & bin labels (Per Student Budget: min=578, max=655)
bin_breakpoints = [0, 584.9, 609.9, 634.9, 659.9]
bin_labels = ["<$585", "$585-610", "$610-635", "$635-660"]

In [40]:
#Adds the new series above (cut_series) to the current df as a new column
school_spending_df["School Spending (per Student)"] = pd.cut(
    x=cut_series, 
    bins=bin_breakpoints, 
    labels=bin_labels, 
    include_lowest=True
)

In [41]:
#Creates df grouped by average budget per student
avg_budget_df = school_spending_df.groupby(["School Spending (per Student)"]).mean()

#Fixes formatting
avg_budget_df['Average Math Score'] = avg_budget_df['Average Math Score'].map('{:.2f}'.format)
avg_budget_df['Average Reading Score'] = avg_budget_df['Average Reading Score'].map('{:.2f}'.format)
avg_budget_df['% Math Passing'] = avg_budget_df['% Math Passing'].map('{:.2f}'.format)
avg_budget_df['% Reading Passing'] = avg_budget_df['% Reading Passing'].map('{:.2f}'.format)
avg_budget_df['% Overall Passing'] = avg_budget_df['% Overall Passing'].map('{:.2f}'.format)

avg_budget_df

,Average Math Score,Average Reading Score,% Math Passing,% Reading Passing,% Overall Passing
School Spending (per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-610,83.60,83.89,94.23,95.90,90.22
$610-635,80.20,82.43,80.04,89.54,72.62
$635-660,77.87,81.37,70.35,83.00,58.86


## Scores by School Size

In [42]:
#Creates a series that contains the list of bin data (Total Students)
cut_series2 = school_summary_df["Total Students"]

#list of breakpoints or bins to fill & labels
bin_ranges = [0, 999.9, 1999.9, 4999.9]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [43]:
#Adds the new series above (cut_series2) to the current df as a new column
school_summary_df["School Size"] = pd.cut(
    x=cut_series2, 
    bins=bin_ranges, 
    labels=size_labels,
    include_lowest=True
)

<ipython-input-43-0de9776bbcf5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_summary_df["School Size"] = pd.cut(


In [44]:
#Creates df grouped by average school size
avg_schoolsize_df = school_summary_df.groupby(["School Size"]).mean()

In [45]:
avg_schoolsize_df.drop(["Total Students"], axis=1)

,Average Math Score,Average Reading Score,% Math Passing,% Reading Passing,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [46]:
#Creates df grouped by type of school
school_type = school_summary_df.groupby(["School Type"]).mean()

In [47]:
#Deletes column not needed for summary
school_type.drop(["Total Students"], axis=1)

,Average Math Score,Average Reading Score,% Math Passing,% Reading Passing,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
